In [7]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from openmmforcefields.generators import EspalomaTemplateGenerator, GAFFTemplateGenerator, SMIRNOFFTemplateGenerator
from openff.toolkit import Molecule
import parmed
import pdbfixer

In [8]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')


In [9]:
import json
from cosolvkit.parametrize import _parametrize_cosolvents

In [10]:
forcefield = _parametrize_cosolvents(cosolvents=["c1ccccc1"],
                                     forcefield=forcefield,
                                     small_molecule_ff="espaloma")

In [11]:
fixer = pdbfixer.PDBFixer(filename="cosolv_system_protein.pdb")
fixer.addMissingHydrogens(7.0)
PDBFile.writeFile(fixer.topology, fixer.positions, open("cosolv_system_protein_clean.pdb", 'w'))
cosolvent_pdb = PDBFile("cosolv_system_protein_clean.pdb")
# cosolvent_pdb = PDBFile("cosolv_system_protein.pdb")
# cosolvent_pdb = PDBFile("output.pdb")

/home/nick/miniconda3/envs/cosolvkit/lib/python3.11/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (ATOM    561   N  VAL A  73      -0.452 -24.313  15.218  0.00  0.00           N  , ATOM    560   N  VAL A  73      -0.486 -24.361  15.155  0.00  0.00           N  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/nick/miniconda3/envs/cosolvkit/lib/python3.11/site-packages/openmm/app/internal/pdbstructure.py:537: UserWarning: WARNING: duplicate atom (ATOM    563  CA  VAL A  73       0.444 -25.499  15.370  0.00  0.00           C  , ATOM    562  CA  VAL A  73       0.447 -25.504  15.399  0.00  0.00           C  )
  warnings.warn("WARNING: duplicate atom (%s, %s)" % (atom, old_atom._pdb_string(old_atom.serial_number, atom.alternate_location_indicator)))
/home/nick/miniconda3/envs/cosolvkit/lib/python3.11/site-packages/openmm/app/internal/pdbstructure

In [12]:
system = forcefield.createSystem(cosolvent_pdb.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=12 * angstrom,
                                 constraints=HBonds,
                                 hydrogenMass=3 * amu)

Did not recognize residue THR; did you forget to call .add_molecules() to add it?


ValueError: No template found for residue 304 (THR).  The set of atoms matches THR, but the bonds are different.  Perhaps the chain is missing a terminal group?  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template